In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from pprint import pprint
import matplotlib.pyplot as plt
from hmmlearn.hmm import CategoricalHMM

In [ ]:
states = ['Kitchen', 'Bedroom', 'Living Room']
hidden_states = ['Cooking', 'Sleeping', 'Watching TV']

In [ ]:
pi_states = [0.4, 0.3, 0.3]
pi_hidden = [0.1, 0.2, 0.7]

In [ ]:
state_space = pd.Series(pi_states, index=states)
hidden_state_space = pd.Series(pi_hidden, index=hidden_states)

In [ ]:
q_df = pd.DataFrame([[0.4, 0.2, 0.4], [0.45, 0.45, 0.1], [0.45, 0.25, 0.3]], 
                    columns=states, index=states)
a_df = pd.DataFrame([[0.3, 0.5, 0.2], [0.1, 0.7, 0.2], [0.2, 0.3, 0.5]], 
                    columns=hidden_states, index=hidden_states)
b_df = pd.DataFrame([[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.2, 0.1, 0.7]], 
                    columns=states, index=hidden_states)

In [ ]:
q_df

In [ ]:
a_df

In [ ]:
b_df

In [ ]:
q = q_df.values
a = a_df.values
b = b_df.values

In [ ]:
def add_edges(df):
    return {(idx, col): df.loc[idx, col] for idx in df.index for col in df.columns}

In [ ]:
edge_wts = add_edges(q_df)
hidden_edge_wts = add_edges(a_df)
emit_edge_wts = add_edges(b_df)

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(states + hidden_states)

for k, v in {**hidden_edge_wts, **emit_edge_wts}.items():
    G.add_edge(k[0], k[1], weight=v, label=f"{v:.2f}")

pos = nx.circular_layout(G)
node_colors = ['skyblue' if node in hidden_states else 'lightgreen' for node in G]
nx.draw(G, pos, with_labels=True, arrows=True, node_color=node_colors, node_size=1000, font_size=5, font_weight='bold')
nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'label'), font_size=8)
plt.show()

In [ ]:
obs_seq = ['kitchen', 'bedroom', 'living room', 'kitchen', 'bedroom']
obs_map = {'kitchen': 0, 'bedroom': 1, 'living room': 2}
obs_idx = np.array([obs_map[o.lower()] for o in obs_seq]).reshape(-1, 1)

model = CategoricalHMM(n_components=len(hidden_states))
model.startprob_ = pi_hidden
model.transmat_ = a
model.emissionprob_ = b

logprob, path = model.decode(obs_idx, algorithm="viterbi")
state_map = {0: 'Cooking', 1: 'Sleeping', 2: 'Watching TV'}
state_path = [state_map[v] for v in path]

# Display result
result = pd.DataFrame({'Observation': obs_seq, 'Best_path': state_path})
print(result)